# Python Regular Expressions
<sup><sub>Adattato da: [Google for Education - Python](https://developers.google.com/edu/python/regular-expressions)</sup></sub>

Le espressioni regolari sono uno strumento altamente efficace e flessibile per
trovare pattern all'interno di sequenze testuali.

In questo tutorial introduttivo, vedremo come implementare e usare le epsressioni regolari in *Python*,
ma li stessi principi possono essere applicati indipendentemente dal linguaggio,
e anche in molti tool che si usano quotidianamente (e.g ricerca in windows, word, ecc.).

### Librerie
Libreria base di python: ```import re```

#### Alternative:
- Regex: Retrocompatibile con re, ha qualche chicca in più, per i più esigenti.
```
pip install regex
import regex
```

In [1]:
#@title Import all libraries
import re


In Python, la sintassi di base per fare match della regex su un testo è:
```python
match = re.search(pattern, string)
```

Ad esempio:
```python
string = 'an example word:cats!!'
match = re.search(r'word:\w\w\w', string)
```
Le regole di base del matching delle regex sono:
    - La ricerca va dall'inizio alla fine dell sequenza, fermandosi al primo match (se non specificato difersamente
    chiamando altri metodi, e.g. `findall()`)
    - Tutto il pattern deve essere trovato
    - Se `match = re.search(pat, str)` non ha successo, match é `None`, altrimenti `match.group()` e il risultato.

NOTA:

Il prefisso `r` prima del pattern della regex indica a python di trattare la striga come "raw", ovvero non modificare il
contenuto (e.g. fare l'escape di caratteri quali il backslash).

In [2]:
str = 'an example word:cats!!'
match = re.search(r'word:\w\w\w', str)

In [ ]:
# If-statement after search() tests if it succeeded
if match:
  print('Trovato', match.group()) ## 'found word:cat'
else:
  print('Non trovato')

Python ha due primitive per la ricerca
`re.match()` - Match solo all'inizio della stringa
`re.search()` - Match all'interno di tutta la stringa

In [ ]:
res = re.match("c", "abcdef")
assert res is None
res = re.search("c", "abcdef")
assert res is not None

## Esempi:

In [ ]:
#@title Base
## Search for pattern 'iii' in string 'piiig'.
## All of the pattern must match, but it may appear anywhere.
## On success, match.group() is matched text.
match = re.search(r'iii', 'piiig')
assert match.group() == "iii"
match = re.search(r'igs', 'piiig') # not found, match == None
assert match == None

In [2]:
#@title Caratteri Speciali
## . = any char but \n
match = re.search(r'..g', 'piiig')
assert match.group() == "iig"


## \d = carattere numerico, \w = carattere alfabeto (minuscolo)
match = re.search(r'\d\d\d', 'p123g') #
assert match.group() == "123"
match = re.search(r'\w\w\w', '@@abcdEFG!!')
assert match.group() == "abc"

In [ ]:
#@title Word bound

# Matchare la stringa 'foo' facendo attenzione che non sia sottostringa di un'altra parola

foo = re.compile(r'\bfoo\b')
text_success = ['foo', 'foo. bar', 'bar.foo', '(foo)']
text_fail = ['foobar', 'bafoon', 'food']
succes = [foo.search(text) is not None for text in text_success]
fail = [foo.search(text) is None for text in text_fail]

assert all(succes)
assert all(fail)

In [ ]:
tweet = '''Good advice! RT @TheNextWeb: What I would do differently if I was learning to code today http://t.co/lbwej0pxOd cc: @garybernhardt #rstats'''

desired_output = 'Good advice What I would do differently if I was learning to code today'

def clean_tweet(tweet):
    tweet = re.sub('http\S+\s*', '', tweet)  # remove URLs
    tweet = re.sub('RT|cc', '', tweet)  # remove RT and cc
    tweet = re.sub('#\S+', '', tweet)  # remove hashtags
    tweet = re.sub('@\S+', '', tweet)  # remove mentions
    punctuation = re.escape("""!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~""")
    tweet = re.sub(f'[{punctuation}]', '', tweet)  # remove punctuations
    tweet = re.sub('\s+', ' ', tweet)  # remove extra whitespace
    return tweet

print(clean_tweet(tweet))

### Argomenti Avanzati

In [2]:
#@title Gruppi Denominati
import re
dt1 = "2026-12-02"
pattern = re.compile(r'(?P<year>\d{4})-(?P<month>\d{1,2})-(?P<day>\d{1,2})')
res = pattern.sub(r"\g<day>-\g<month>-\g<year>", dt1)
assert res == '02-12-2026'

In [ ]:
#@title Backreferencing

Dato una string che contiene due informationi:
1 - Info base impiegato
Inizia con @nome e finisce con nome
es: @daniel dxc chennai 45000 male daniel
2 - Posizione impiegato
Inizia con %nome e finisce con none%
es: %daniel python developer daniel%

In [13]:
str="""
@daniel dxc chennai 45000 male daniel @henry infosys bengaluru 29000 male hobby-
swimming henry
@raja zoho chennai 37000 male raja @ramu infosys bengaluru 99000 male hobby-badminton
ramu
%daniel python developer daniel% %henry database admin henry%
%raja Testing lead raja% %ramu Manager ramu%
"""

#backreferencing employee name (\w+)  <----  \1
#----------------------------------------------
basic_info=re.findall(r'@+(\w+)\s(.*)\s\1',str, re.MULTILINE)
print(basic_info)

#(%) <-- \1  and (\w+) <--- \2
#-------------------------------
designation=re.findall(r'(%)+(\w+)(.*?)\2\1',str)
print(designation)

for i in range(len(designation)):
    designation[i]=(designation[i][1],designation[i][2])
print(designation)

[('daniel', 'dxc chennai 45000 male'), ('raja', 'zoho chennai 37000 male'), ('ramu', 'infosys bengaluru 99000 male hobby-badminton')]
[('%', 'daniel', ' python developer '), ('%', 'henry', ' database admin '), ('%', 'raja', ' Testing lead '), ('%', 'ramu', ' Manager ')]
[('daniel', ' python developer '), ('henry', ' database admin '), ('raja', ' Testing lead '), ('ramu', ' Manager ')]


In [26]:
#@title Lookahead/Lookbehind
# Trovare i prezzi all'interno della pagina HTML
# il formato e': €50

with open("fb_marketplace.html", "rt", encoding="utf8") as inf:
    webpage = ' '.join(inf.readlines())

def find_prices(text):
    prices = re.findall(r'(?<=€)(\d+)',text)
    return prices

find_prices(webpage)
prices = [int(x) for x in find_prices(webpage)]
media = sum(prices)/len(prices)
print(media)



['50', '50', '50', '50', '30', '30', '10', '10', '60', '60', '150', '150', '70', '70', '5', '5', '1', '1', '535', '535', '123', '123', '30', '30', '30', '30', '75', '75', '20', '20', '30', '30']
